In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

#URL der Webseite
url = "https://pokemondb.net/pokedex/all"

#HTTP-Anfrage senden
response = requests.get(url)
response.raise_for_status()  # Fehler auslösen, falls die Anfrage fehlschlägt

#HTML-Inhalt der Seite parsen
soup = BeautifulSoup(response.text, 'html.parser')

#Tabelle finden
table = soup.find('table', {'id': 'pokedex'})

#Tabellenüberschriften extrahieren
headers = [header.text for header in table.find('thead').find_all('th')]

#Tabelleninhalte extrahieren
rows = []
for row in table.find('tbody').find_all('tr'):
    cells = row.find_all(['th', 'td'])
    rows.append([cell.text.strip() for cell in cells])

#DataFrame erstellen
df = pd.DataFrame(rows, columns=headers)

#Ausgabe des DataFrames
print(df.head())

#Optional: DataFrame in eine CSV-Datei speichern
df.to_csv("pokemon_data.csv", index=False)


      #                    Name          Type Total  HP Attack Defense  \
0  0001               Bulbasaur  Grass Poison   318  45     49      49   
1  0002                 Ivysaur  Grass Poison   405  60     62      63   
2  0003                Venusaur  Grass Poison   525  80     82      83   
3  0003  Venusaur Mega Venusaur  Grass Poison   625  80    100     123   
4  0004              Charmander          Fire   309  39     52      43   

  Sp. Atk Sp. Def Speed  
0      65      65    45  
1      80      80    60  
2     100     100    80  
3     122     120    80  
4      60      50    65  


In [15]:
import pandas as pd  # Stelle sicher, dass pandas importiert ist

# Datei laden
pokemon_data = pd.read_csv('pokemon_data.csv')

# 1. Die stärksten Pokémon (für jeden Typ)
strongest_pokemon = pokemon_data.loc[pokemon_data.groupby("Type")["Total"].idxmax(), ["Type", "Name", "Total"]]

# 2. Die besten Angreifer
best_attackers = pokemon_data.sort_values(by="Attack", ascending=False).head(10)[["Name", "Attack"]]

# 3. Durchschnittliche Werte für jeden Typ
average_stats_by_type = pokemon_data.groupby("Type").mean(numeric_only=True).reset_index()

# Ergebnisse anzeigen
print("Stärkste Pokémon (für jeden Typ):")
print(strongest_pokemon)

print("\nBeste Angreifer:")
print(best_attackers)

print("\nDurchschnittswerte für jeden Typ:")
print(average_stats_by_type)


Stärkste Pokémon (für jeden Typ):
               Type                      Name  Total
167             Bug                    Pinsir    500
1095       Bug Dark                     Lokix    450
888    Bug Electric                  Vikavolt    500
896       Bug Fairy                  Ribombee    464
274    Bug Fighting  Heracross Mega Heracross    600
...             ...                       ...    ...
176       Water Ice                    Lapras    535
94     Water Poison                Tentacruel    515
108   Water Psychic      Slowbro Mega Slowbro    590
687      Water Rock                Carracosta    495
489     Water Steel                  Empoleon    530

[221 rows x 3 columns]

Beste Angreifer:
                         Name  Attack
201      Mewtwo Mega Mewtwo X     190
274  Heracross Mega Heracross     185
956                   Kartana     181
475    Rayquaza Mega Rayquaza     180
478       Deoxys Attack Forme     180
473    Groudon Primal Groudon     180
545    Garchomp Mega G